# Three Body Problem: Neural Network Training

A simple neural network is trained to learn approximate solutions to the three body problem.<br>
Data is sampled from three body systems with parameters similar to planets in the solar system.

In [1]:
# Library imports
import tensorflow as tf
import rebound
import numpy as np
import datetime
import matplotlib.pyplot as plt

# Aliases
keras = tf.keras

In [2]:
# Local imports
from utils import load_vartbl, save_vartbl, plot_style, range_inc
from tf_utils import gpu_grow_memory, TimeHistory
from tf_utils import plot_loss_hist, EpochLoss, TimeHistory
from tf_utils import Identity

from orbital_element import OrbitalElementToConfig, ConfigToOrbitalElement, MeanToTrueAnomaly, G_
from orbital_element import make_model_elt_to_cfg, make_model_cfg_to_elt

from jacobi import CartesianToJacobi, JacobiToCartesian

from g3b_data import make_traj_g3b, make_data_g3b, make_datasets_g3b, traj_to_batch
from g3b_data import make_datasets_solar, make_datasets_hard
from g3b_data import combine_datasets_g3b, combine_datasets_solar
from sej_data import load_data_sej, make_datasets_sej, combine_datasets_sej

from g3b_plot import plot_orbit_q, plot_orbit_v, plot_orbit_a, plot_orbit_energy, plot_orbit_element
from g3b import KineticEnergy_G3B, PotentialEnergy_G3B, Momentum_G3B, AngularMomentum_G3B
from g3b import VectorError, EnergyError
from g3b import Motion_G3B, make_physics_model_g3b
from g3b import fit_model
from g3b_model_math import make_position_model_g3b_math, make_model_g3b_math
from g3b_model_nn import make_position_model_g3b_nn, make_model_g3b_nn

In [3]:
# Set active GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_visible_devices(gpus[1:2], 'GPU')

In [4]:
# Grow GPU memory (must be first operation in TF)
gpu_grow_memory()

In [5]:
# Lightweight serialization
fname = '../data/g3b/g3b_train.pickle'
vartbl = load_vartbl(fname)

In [6]:
# Set plot style
plot_style()

### Load Data for General Three Body Problem

In [7]:
# Description of datasets to be loaded
n_years = 100
sample_freq = 10
traj_size = n_years * sample_freq + 1

In [8]:
# Configuration for loading data sets
# num_data_sets = 50
num_data_sets = 5
batch_size = 64
# num_gpus = 1
# full_batch_size = num_gpus * batch_size

# Set size of tiny data sets
n_traj_tiny = batch_size

# Set starting random seed
seed0 = 42

In [9]:
# Create a tiny data set with one batch of solar type orbits
ds_tiny_solar_trn, ds_tiny_solar_val, ds_tiny_solar_tst = \
    make_datasets_solar(n_traj=n_traj_tiny, vt_split=1.0, 
                        n_years=n_years, sample_freq=sample_freq,
                        batch_size=batch_size, seed=seed0)

Loaded data from ../data/g3b/304606075.pickle.


W0821 10:19:59.034000 139762367096640 deprecation.py:323] From /home/michael/anaconda3/envs/nbody/lib/python3.7/site-packages/tensorflow/python/data/util/random_seed.py:58: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
# Build combined solar data sets
# ds_solar_trn, ds_solar_val, ds_solar_tst = \
#     combine_datasets_solar(num_data_sets=num_data_sets, batch_size=batch_size, seed0=seed0)

### Load Data for Perturbed Sun-Earth-Jupiter System

In [11]:
# Orbital perturbation scales
sd_log_a = 0.01
sd_log_e = 0.10
sd_log_inc = 0.10
sd_Omega = np.pi * 0.02
sd_omega = np.pi * 0.02
sd_f = np.pi * 0.02

# Wrap into dictionary
sej_sigma = {
    'sd_log_a': sd_log_a,
    'sd_log_e': sd_log_e,
    'sd_log_inc': sd_log_inc,
    'sd_Omega': sd_Omega,
    'sd_omega': sd_omega,
    'sd_f': sd_f
}

In [12]:
# Create a tiny data set with one batch of perturbed SEJ orbits
ds_tiny_sej_trn, ds_tiny_sej_val, ds_tiny_sej_tst = \
    make_datasets_sej(n_traj=n_traj_tiny, vt_split=1.0, n_years=n_years, sample_freq=sample_freq,
                      **sej_sigma,
                      batch_size=batch_size, seed=seed0)

Loaded data from ../data/sej/1181713387.pickle.


In [14]:
# Create dictionary for unperturbed orbits
sej_sigma0 = {k: v*0.0 for k, v in sej_sigma.items()}

# Create a tiny data set with the unperturbed SEJ system
ds_sej0, _, _ = \
    make_datasets_sej(n_traj=n_traj_tiny, vt_split=0.0, n_years=n_years, sample_freq=sample_freq,
                      **sej_sigma0,
                      batch_size=batch_size, seed=0)

Loaded data from ../data/sej/2974512314.pickle.


In [15]:
# Check that starting positions identical to roundoff
inputs, outputs = list(ds_sej0.take(1))[0]
np.std(inputs['q0'], axis=(0,))
# inputs['q0'][1]-inputs['q0'][0]

array([[2.3283064e-10, 0.0000000e+00, 2.9103830e-11],
       [5.9604645e-08, 4.7683716e-07, 2.9103830e-11],
       [4.7683716e-07, 9.5367432e-07, 2.9802322e-08]], dtype=float32)

In [16]:
# Check that calculated positions essentially identical
np.sqrt(np.mean(np.square(np.std(outputs['q'], axis=(0,)))))

7.999339e-07

In [17]:
np.linalg.norm(outputs['q'][19]-outputs['q'][17])

0.0

In [ ]:
# Build combined SEJ data sets
# ds_sej_trn, ds_sej_val, ds_sej_tst = \
#     combine_datasets_solar(num_data_sets=num_data_sets, batch_size=batch_size, seed0=seed0)

In [18]:
# Alias ds_trn, ds_val, ds_tst to selected source

# The selected data type for this analysis
data_type = 'solar'

# Tables mapping data type to tuple of data sets
data_tiny_by_type = {
    'solar': (ds_tiny_solar_trn, ds_tiny_solar_val, ds_tiny_solar_tst),
    'SEJ': (ds_tiny_sej_trn, ds_tiny_sej_val, ds_tiny_sej_tst)
}

data_by_type = {
    # 'solar': (ds_solar_trn, ds_solar_val, ds_solar_tst),
    #'SEJ': (ds_sej_trn, ds_sej_val, ds_sej_tst)
}

# Perform the aliasing
ds_tiny_trn, ds_tiny_val, ds_tiny_tst = data_tiny_by_type[data_type]
# ds_trn, ds_val, ds_tst = data_by_type[data_type]

### Create the Math Model as a Benchmark

In [19]:
model_math = make_model_g3b_math(traj_size=traj_size, batch_size=batch_size)

In [20]:
optimizer = keras.optimizers.Adam(learning_rate=0.0)

loss = {'q': VectorError(name='q_loss'),
        'v': VectorError(name='v_loss'),
        'a': VectorError(regularizer=1.0, name='a_loss'),
        'q0_rec': VectorError(name='q0_loss'),
        'v0_rec': VectorError(name='v0_loss'),
        'H': EnergyError(name='H_loss'),
        'P': VectorError(name='P_loss', regularizer=1.0),
        'L': VectorError(name='L_loss'),
       }

metrics = None

loss_weights = {'q': 1.0,
                'v': 1.0,
                'a': 1.0,
                'q0_rec': 1.0E4,
                'v0_rec': 1.0E4,
                'H': 1.0,
                'P': 1.0,
                'L': 1.0}

In [21]:
# Compile the full mathematical model
model_math.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

In [23]:
ds_tiny_solar_trn

<BatchDataset shapes: ({t: (64, 1001), q0: (64, 3, 3), v0: (64, 3, 3), m: (64, 3)}, {q: (64, 1001, 3, 3), v: (64, 1001, 3, 3), a: (64, 1001, 3, 3), orb_a: (64, 1001, 2), orb_e: (64, 1001, 2), orb_inc: (64, 1001, 2), orb_Omega: (64, 1001, 2), orb_omega: (64, 1001, 2), orb_f: (64, 1001, 2), q0_rec: (64, 3, 3), v0_rec: (64, 3, 3), T: (64, 1001), U: (64, 1001), H: (64, 1001), P: (64, 1001, 3), L: (64, 1001, 3)}), types: ({t: tf.float32, q0: tf.float32, v0: tf.float32, m: tf.float32}, {q: tf.float32, v: tf.float32, a: tf.float32, orb_a: tf.float32, orb_e: tf.float32, orb_inc: tf.float32, orb_Omega: tf.float32, orb_omega: tf.float32, orb_f: tf.float32, q0_rec: tf.float32, v0_rec: tf.float32, T: tf.float32, U: tf.float32, H: tf.float32, P: tf.float32, L: tf.float32})>

In [24]:
ds_tiny_trn

<BatchDataset shapes: ({t: (64, 1001), q0: (64, 3, 3), v0: (64, 3, 3), m: (64, 3)}, {q: (64, 1001, 3, 3), v: (64, 1001, 3, 3), a: (64, 1001, 3, 3), orb_a: (64, 1001, 2), orb_e: (64, 1001, 2), orb_inc: (64, 1001, 2), orb_Omega: (64, 1001, 2), orb_omega: (64, 1001, 2), orb_f: (64, 1001, 2), q0_rec: (64, 3, 3), v0_rec: (64, 3, 3), T: (64, 1001), U: (64, 1001), H: (64, 1001), P: (64, 1001, 3), L: (64, 1001, 3)}), types: ({t: tf.float32, q0: tf.float32, v0: tf.float32, m: tf.float32}, {q: tf.float32, v: tf.float32, a: tf.float32, orb_a: tf.float32, orb_e: tf.float32, orb_inc: tf.float32, orb_Omega: tf.float32, orb_omega: tf.float32, orb_f: tf.float32, q0_rec: tf.float32, v0_rec: tf.float32, T: tf.float32, U: tf.float32, H: tf.float32, P: tf.float32, L: tf.float32})>

In [22]:
# model_math.evaluate(ds_tiny_trn)
model_math.evaluate(ds_tiny_solar_trn)

1/1 [==============================] - 2s 2s/step - loss: 0.0161 - q_loss: 0.0094 - v_loss: 0.0049 - a_loss: 0.0017 - q0_rec_loss: 3.4913e-14 - v0_rec_loss: 2.9723e-14 - H_loss: 3.4942e-09 - P_loss: 6.2723e-21 - L_loss: 2.0751e-14


[0.016079716384410858,
 0.009423083,
 0.004922431,
 0.0017341995,
 3.4913343e-14,
 2.9722542e-14,
 3.4942307e-09,
 6.2723486e-21,
 2.075063e-14]

In [ ]:
model_math.evaluate(ds_tiny_trn)

### Train the Neural Network Model

In [ ]:
# Configuration for neural network model architecture
# hidden_sizes = [64, 16]
hidden_sizes = []
skip_layers = True
traj_size = 1001

# Training configuration
reg = 1.0E2
kernel_reg = reg
activity_reg = reg
learning_rate = 1.0E-4

In [ ]:
# Build neural network model
model_nn = make_model_g3b_nn(hidden_sizes=hidden_sizes, skip_layers=skip_layers, 
                             kernel_reg=kernel_reg, activity_reg=activity_reg,
                             traj_size=traj_size, batch_size=batch_size)

In [ ]:
# model_nn.summary()

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
# optimizer = keras.optimizers.Adagrad(learning_rate=learning_rate)
# optimizer = keras.optimizers.Adadelta()

loss = {'q': VectorError(name='q_loss'),
        'v': VectorError(name='v_loss'),
        'a': VectorError(regularizer=1.0E-2, name='a_loss'),
        'q0_rec': VectorError(name='q0_loss'),
        'v0_rec': VectorError(name='v0_loss'),
        'H': EnergyError(name='H_loss'),
        'P': VectorError(name='P_loss', regularizer=1.0E-6),
        'L': VectorError(name='L_loss'),
       }

metrics = None

loss_weights = {'q': 1.0,
                'v': 1.0,
                'a': 1.0,
                'q0_rec': 1.0E4,
                'v0_rec': 1.0E4,
                'H': 1.0,
                'P': 1.0,
                'L': 1.0}

In [ ]:
# Compile the NN model
model_nn.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

In [ ]:
# Evaluate the NN model on the validation data
model_nn.evaluate(ds_val)

In [ ]:
# Compare this to math model - should be the same before training
# model_math.evaluate(ds_val)

# Hard coded baseline losses
loss_baseline_list = \
[0.01564649583971394,
 0.0051587406,
 0.0053154626,
 0.005172265,
 3.3106947e-14,
 3.2552384e-14,
 2.5221508e-08,
 1.1691268e-14,
 2.1020908e-14]

# Baseline position loss
q_loss_baseline = loss_baseline_list[1]

# Table of baseline losses
keys = ['loss', 'q_loss', 'v_loss', 'a_loss', 'q0_rec_loss', 'v0_rec_loss', 'H_loss', 'P_loss', 'L_loss']
loss_baseline = {key: loss_baseline_list[i] for i, key in enumerate(keys)}
# Set dummy batch_num and time
loss_baseline['batch_num'] = 0
loss_baseline['time'] = 0.0

# Initialize history before training
hist0 = {key: np.array([val], dtype=np.float32) for key, val in loss_baseline.items()}

# Review baseline loss table
# loss_baseline

In [ ]:
# Set up training
suffix = '_'.join(str(sz) for sz in hidden_sizes)
model_name = f'model_g3b_nn_{suffix}'
model_h5 = f'../models/g3b/{model_name}.h5'
hist_name = model_name.replace('model_', 'hist_')
epochs = 1
save_freq = 'epoch'

In [ ]:
# Attempt to load model or train a single epoch
try:
    model_nn.load_weights(model_h5)
    model_nn.compile(loss=loss, optimizer=optimizer, metrics=metrics, loss_weights=loss_weights)
    hist = vartbl[hist_name]
    print(f'Loaded {model_name} from {model_h5}.')
except:
    print(f'Unable to load {model_name} from {model_h5}. Fitting...')
    hist = fit_model(model=model_nn, 
                     ds=ds_trn, 
                     epochs=epochs,
                     save_freq=save_freq,
                     prev_history = hist0, 
                     batch_num=1)
    vartbl[hist_name] = hist
    save_vartbl(vartbl, fname)

In [ ]:
# num_epochs = 50
num_epochs = 1
for i in range_inc(1, num_epochs):
    ts = datetime.datetime.now()
    st = ts.strftime('%Y-%m-%d %H:%M:%S')
    print(f'*** Training loop {i:3} *** - {st}')
    hist = fit_model(model=model_nn, 
                     ds=ds_trn, 
                     epochs=epochs,
                     loss=loss, 
                     optimizer=optimizer,
                     metrics=metrics,
                     save_freq=save_freq,
                     prev_history = hist, 
                     batch_num=i)
    vartbl[hist_name] = hist
    save_vartbl(vartbl, fname)

In [ ]:
# Plot the loss
fig, ax = plot_loss_hist(hist=hist, model_name=model_nn.name, key='q_loss', baseline=q_loss_baseline)

In [ ]:
# Plot the total loss
fig, ax = plot_loss_hist(hist=hist, model_name=model_nn.name, key='loss')

In [ ]:
hist

In [ ]:
# Evaluate the trained model on the training data
# model_nn.evaluate(ds_trn)

In [ ]:
# Evaluate the trained model on the test data
# model_nn.evaluate(ds_tst)

## Perturbed Sun-Earth-Jupiter System

In [ ]:
from sej_data import load_data_sej, make_datasets_sej, combine_datasets_sej
import numpy as np
from g3b_plot import plot_orbit_q, plot_orbit_v, plot_orbit_a, plot_orbit_energy, plot_orbit_element
import matplotlib.pyplot as plt

In [ ]:
# Trajectory length
n_years = 100
sample_freq = 10

# Number of trajectories
num_batches = 1
n_traj = 100
vt_split = 0.20
batch_size = 64

In [ ]:
# Orbital perturbation scales
sd_log_a = 0.01
sd_log_e = 0.10
sd_log_inc = 0.10
sd_Omega = np.pi * 0.02
sd_omega = np.pi * 0.02
sd_f = np.pi * 0.02

In [ ]:
# List of seeds to use for datasets
seed0 = 42
seed1 = seed0 + num_batches * 3
seeds = list(range(seed0, seed1, 3))

In [ ]:
data = load_data_sej(n_traj=n_traj, vt_split=vt_split, n_years=n_years, sample_freq=sample_freq,
                     sd_log_a=sd_log_a, sd_log_e=sd_log_e, sd_log_inc=sd_log_inc,
                     sd_Omega=sd_Omega, sd_omega=sd_omega, sd_f=sd_f, seed=seed0)

In [ ]:
inputs_trn, outputs_trn = data[0:2]

data_trn = {**inputs_trn, **outputs_trn}

In [ ]:
inputs_trn['t'].shape

In [ ]:
outputs_trn['q'].shape

In [ ]:
plt.plot(inputs_trn['t'][0], outputs_trn['q'][0][:, 1, 0])

In [ ]:
plt.plot(inputs_trn['t'][0], outputs_trn['q'][0][:, 1, 1])

In [ ]:
np.mean(outputs_trn['orb_a'], axis=(0))

In [ ]:
outputs_trn['orb_a'].shape

In [ ]:
np.mean(outputs_trn['orb_a'][:, 0, :], axis=0)

In [ ]:
np.std(outputs_trn['orb_a'][:, 0, :], axis=0)

In [ ]:
np.mean(outputs_trn['orb_e'][:, 0, :], axis=0)

In [ ]:
np.std(outputs_trn['orb_e'][:, 0, :], axis=0)